sentiment analysis

In [16]:
!pip install transformers --quiet
!pip install --upgrade transformers --quiet


In [2]:
!pip install onnx  --quiet
!pip install torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 34.3 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
import torch
def analyze_sentiment(text):
    nlp = pipeline('sentiment-analysis', model='distilroberta-base')
    result = nlp(text)
    sentiment_label = result[0]['label']
    return sentiment_label

# Example usage:
text_to_analyze = "I love this movie! It's so exciting and entertaining."
sentiment = analyze_sentiment(text_to_analyze)
print(f"Sentiment: {sentiment}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment: LABEL_1


In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

def analyze_sentiment(text):
    model_name = 'distilroberta-base'
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaForSequenceClassification.from_pretrained(model_name)
    # Preprocess the text and tokenize it
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted sentiment label
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).tolist()[0]
    sentiment_labels = ['negative', 'positive']
    predicted_label = sentiment_labels[probabilities.index(max(probabilities))]

    return predicted_label


In [5]:
def quantize_model(model, data_loader):
    model.eval()
    model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
    model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

    # Run the model on the data to trigger quantization
    with torch.no_grad():
        for batch in data_loader:
            _ = model(**batch)

    return model


*Function  to Perform Sentiment Analysis:*

In [6]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

def analyze_sentiment(text):
    model_name = 'distilroberta-base'
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    model = RobertaForSequenceClassification.from_pretrained(model_name)

    # Preprocess the text and tokenize it
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted sentiment label
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1).tolist()[0]
    sentiment_labels = ['negative', 'positive']
    predicted_label = sentiment_labels[probabilities.index(max(probabilities))]

    return predicted_label


*Function to Quantize the Model:*

In [7]:
def quantize_model(model):
    # Set the model to evaluation mode for quantization
    model.eval()

    # Quantize the model using post-training static quantization
    quantized_model = torch.quantization.quantize_dynamic(
        model,
        {torch.nn.Linear},
        dtype=torch.qint8
    )

    return quantized_model


*Example*

In [8]:
import time

In [9]:
# Prepare a dummy dataset for demonstration purposes
dummy_dataset = ["I love this movie!", "This movie is terrible!", "The acting was great."]

# Tokenize and preprocess the dummy dataset
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
dummy_inputs = tokenizer(dummy_dataset, padding=True, truncation=True, return_tensors='pt')

# Load the original model
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base')

# Measure inference speed before quantization
start_time = time.time()
with torch.no_grad():
    outputs = model(**dummy_inputs)  # Pass the entire inputs dictionary at once
end_time = time.time()
inference_time_before_quantization = end_time - start_time

# Quantize the model
quantized_model = quantize_model(model)

# Save the quantized model to a file
output_file = "./quantized_model.pt"
torch.save(quantized_model.state_dict(), output_file)
print("Quantized model has been saved.")


# Measure inference speed after quantization
start_time = time.time()
with torch.no_grad():
    outputs = quantized_model(**dummy_inputs)  # Pass the entire inputs dictionary at once
end_time = time.time()
inference_time_after_quantization = end_time - start_time

# Print the inference times
print(f"Inference Time Before Quantization: {inference_time_before_quantization:.6f} seconds")
print(f"Inference Time After Quantization: {inference_time_after_quantization:.6f} seconds")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Quantized model has been saved.
Inference Time Before Quantization: 0.205621 seconds
Inference Time After Quantization: 0.121297 seconds


#####################

Rough Work

In [10]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the DistilRoberta-Base model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# Input text
text = "This is an example sentence for testing."

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt')

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the logits and probabilities
logits = outputs.logits
probabilities = torch.softmax(logits, dim=1).tolist()[0]

# Print the probabilities for each class
print("Probabilities for each class:")
print(f"Positive: {probabilities[1]:.4f}")
print(f"Negative: {probabilities[0]:.4f}")

# Export the model to ONNX format
input_names = ["input_ids", "attention_mask"]
output_names = ["logits"]
dummy_input = torch.zeros(1, 512, dtype=torch.long)  # Adjust the input size based on your use case
torch.onnx.export(model, (inputs['input_ids'], inputs['attention_mask']), "distilroberta-base.onnx", verbose=True, input_names=input_names, output_names=output_names)

print("DistilRoberta-Base model has been exported to ONNX format.")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Probabilities for each class:
Positive: 0.5131
Negative: 0.4869
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

DistilRoberta-Base model has been exported to ONNX format.


**QUANTISING ONNX MODEL**

In [11]:
#!pip install onnxruntime  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.0 MB/s eta 0:00:00


quantising

In [12]:
#!pip install onnxruntime-tools  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.9 MB/s eta 0:00:00


In [13]:
#!pip install transformers torch onnx onnxruntime  --quiet

In [14]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# Load the original ONNX model
onnx_model_path = "distilroberta-base.onnx"
model = onnx.load(onnx_model_path)

# Define the output path for the quantized model
quantized_model_path = "distilroberta-base-dynamic-quantized.onnx"

# Quantize the model dynamically
quantize_dynamic(onnx_model_path, quantized_model_path, weight_type=QuantType.QUInt8)

print(f"The model has been quantized dynamically and saved at {quantized_model_path}.")


Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

testing

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import onnxruntime
from onnxruntime.quantization import quantize_dynamic, QuantType

# Load the PyTorch model and tokenizer
model_name = "distilroberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# An example input
input_ids = tokenizer.encode("This is a sample input", return_tensors='pt')

# Define names and dynamic axes for inputs/outputs
input_names = ['input_ids', 'attention_mask']
output_names = ['output']
dynamic_axes = {'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
                'output': {0: 'batch_size'}}

# Export the model
torch.onnx.export(model,
                  (input_ids, input_ids),
                  "distilroberta-base.onnx",
                  input_names=input_names,
                  output_names=output_names,
                  dynamic_axes=dynamic_axes)

# Quantize the model dynamically
quantize_dynamic("distilroberta-base.onnx",
                 "distilroberta-base-dynamic-quantized.onnx",
                 weight_type=QuantType.QUInt8)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore 